In [ ]:
from langchain.chains import RetrievalQA  # 检索QA链，在文档上进行检索
# from langchain.chat_models import ChatOpenAI  # openai模型
# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain.document_loaders import CSVLoader  # 文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch  # 向量存储

from IPython.display import display, Markdown  # 在jupyter显示信息的工具
import pandas as pd

file = "../data/OutdoorClothingCatalog_1000.csv"

# 使用langchain文档加载器对数据进行导入
loader = CSVLoader(file_path=file)

# 使用pandas导入数据，用以查看
data = pd.read_csv(file, usecols=[1, 2])
data.head()

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


In [ ]:
llm = ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型